# Import and helper functions

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt

def plot_images(green_masked_green, red_masked_green, green_masked_red, red_masked_red, method, base_name):
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    fig.suptitle(f"Masked Images for {base_name} ({method})")

    axs[0, 0].imshow(green_masked_green, cmap='gray')
    axs[0, 0].set_title(f'Green Masked by Green ({method})')

    axs[0, 1].imshow(red_masked_green, cmap='gray')
    axs[0, 1].set_title(f'Red Masked by Green ({method})')

    axs[1, 0].imshow(green_masked_red, cmap='gray')
    axs[1, 0].set_title(f'Green Masked by Red ({method})')

    axs[1, 1].imshow(red_masked_red, cmap='gray')
    axs[1, 1].set_title(f'Red Masked by Red ({method})')

    for ax in axs.flat:
        ax.axis('off')

    plt.show()

def process_masks(green_channel, red_channel, mask, method, base_name):
    # Ensure the mask is 3D and matches the dimensions of the channels
    if mask.ndim != 3 or green_channel.ndim != 3 or red_channel.ndim != 3:
        raise ValueError(f"Incorrect dimensions for {base_name}: Mask or channels are not in the correct format.")

    # Check that the mask and channels have the same height and width
    if green_channel.shape[1:] != mask.shape[1:] or red_channel.shape[1:] != mask.shape[1:]:
        raise ValueError(f"Mask and image dimensions do not match for {base_name}. "
                         f"Green channel: {green_channel.shape}, Mask: {mask.shape}")

    # Normalize the green and red channels to range [0, 1]
    green_channel_normalized = (green_channel - np.min(green_channel)) / (np.max(green_channel) - np.min(green_channel))
    red_channel_normalized = (red_channel - np.min(red_channel)) / (np.max(red_channel) - np.min(red_channel))

    # Apply the mask to each channel
    green_masked_green = green_channel_normalized[0] * mask[0]  # Mask green by green
    red_masked_green = red_channel_normalized[0] * mask[0]      # Mask red by green
    green_masked_red = green_channel_normalized[0] * mask[1]    # Mask green by red
    red_masked_red = red_channel_normalized[0] * mask[1]        # Mask red by red

    # Calculate mean intensities in the masked areas
    intensity_G_masked_G = np.mean(green_masked_green[mask[0] > 0])
    intensity_G_masked_R = np.mean(red_masked_green[mask[0] > 0])
    intensity_R_masked_G = np.mean(green_masked_red[mask[1] > 0])
    intensity_R_masked_R = np.mean(red_masked_red[mask[1] > 0])

    # Calculate J values
    J_G = intensity_G_masked_R / intensity_G_masked_G if intensity_G_masked_G > 0 else 0
    J_R = intensity_R_masked_G / intensity_R_masked_R if intensity_R_masked_R > 0 else 0

    # Print results for debugging
    print(f"{method} Mask Mean Intensities for {base_name}:")
    print(f"Intensity_G_masked_G: {intensity_G_masked_G}")
    print(f"Intensity_G_masked_R: {intensity_G_masked_R}")
    print(f"Intensity_R_masked_G: {intensity_R_masked_G}")
    print(f"Intensity_R_masked_R: {intensity_R_masked_R}")

    print(f"{method} Mask J_G: {J_G}")
    print(f"{method} Mask J_R: {J_R}")

    # Plot the masked images
    plot_images(green_masked_green, red_masked_green, green_masked_red, red_masked_red, method, base_name)

    return J_G, J_R  # Return the J values


def process_images(input_dir, output_csv):
    raw_dir = os.path.join(input_dir, 'raw')
    mask_dir = os.path.join(input_dir, 'masks')

    li_mask_dir = os.path.join(mask_dir, 'Li')
    otsu_mask_dir = os.path.join(mask_dir, 'Otsu')

    # Ask the user once for the mixing type for the entire folder
    mixing_type = input("Is the current batch of images partial or complete mixing? (Enter 'partial' or 'complete'): ").strip().lower()

    # Data to be recorded in the CSV file
    results = []

    # Loop through each image in the raw folder
    for image_name in os.listdir(raw_dir):
        if image_name.endswith(".tif"):
            # Load the raw image
            image_path = os.path.join(raw_dir, image_name)
            raw_image = io.imread(image_path)
            # Raw image is assumed to be (channels, height, width)
            green_channel = raw_image[0:1]  # Extract the green channel as a 3D array
            red_channel = raw_image[1:2]     # Extract the red channel as a 3D array

            # Mask file paths
            li_mask_path = os.path.join(li_mask_dir, f'{image_name}_mask_Li.tif')
            otsu_mask_path = os.path.join(otsu_mask_dir, f'{image_name}_mask_Otsu.tif')

            # Check if the mask files exist
            if not os.path.exists(li_mask_path) or not os.path.exists(otsu_mask_path):
                print(f"Skipping {image_name} as corresponding masks are not found.")
                continue

            # Load the masks
            li_mask = io.imread(li_mask_path)
            otsu_mask = io.imread(otsu_mask_path)

            if mixing_type == 'partial':
                # Process using individual masks - intersection
                intersection_li_mask = np.logical_and(li_mask[0] > 0, li_mask[1] > 0).astype(int)
                intersection_otsu_mask = np.logical_and(otsu_mask[0] > 0, otsu_mask[1] > 0).astype(int)

                nonoverlapping_li_mask_3d = np.stack((li_mask[0] - intersection_li_mask, li_mask[1] - intersection_li_mask), axis=0)
                nonoverlapping_otsu_mask_3d = np.stack((otsu_mask[0] - intersection_otsu_mask, otsu_mask[1] - intersection_otsu_mask), axis=0)
                
                J_G_Li, J_R_Li = process_masks(green_channel, red_channel, nonoverlapping_li_mask_3d, 'Li', image_name)
                J_G_Otsu, J_R_Otsu = process_masks(green_channel, red_channel, nonoverlapping_otsu_mask_3d, 'Otsu', image_name)
           
            elif mixing_type == 'complete':
                # Union of green and red masks for both methods
                union_li_mask = np.logical_or(li_mask[0] > 0, li_mask[1] > 0).astype(int)
                union_otsu_mask = np.logical_or(otsu_mask[0] > 0, otsu_mask[1] > 0).astype(int)

                # Create a 3D mask for processing
                union_li_mask_3d = np.stack((union_li_mask, union_li_mask), axis=0)
                union_otsu_mask_3d = np.stack((union_otsu_mask, union_otsu_mask), axis=0)

                J_G_Li, J_R_Li = process_masks(green_channel, red_channel, union_li_mask_3d, 'Li', image_name)
                J_G_Otsu, J_R_Otsu = process_masks(green_channel, red_channel, union_otsu_mask_3d, 'Otsu', image_name)

            # Calculate the final J values as the mean of Li and Otsu
            J_G = (J_G_Li + J_G_Otsu) / 2
            J_R = (J_R_Li + J_R_Otsu) / 2

            # Append the results to the list
            results.append({
                'Image': image_name,
                'J_G_Li': J_G_Li,
                'J_G_Otsu': J_G_Otsu,
                'J_G': J_G,
                'J_R_Li': J_R_Li,
                'J_R_Otsu': J_R_Otsu,
                'J_R': J_R
            })

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")


# 1:1:1_2 arm_results

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 1:2:1_2 arm_results

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 1:3:1_2 arm_results

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 1:4:1_2 arm_results

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# Troubleshoot for 1:4:1_2 arm

The high mixing index in JG is due to the confusing definition of "mixing"

These condensates have a low green signal intensity within the red signal area. If treated as "partial" mixing, the J is biased towards low range; if treated as "complate" mixing, the J is biased towards high range. 

See results below:

In [ ]:
input_directory = ""

output_csv_complete = ""
process_images(input_directory, output_csv_complete)

output_csv_partial = ""
process_images(input_directory, output_csv_partial)

# 1:5:1_2 arm_results

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 2:1:2_2 arm_results

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 1:1:1_4 arm

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 1:2:1_4 arm

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 1:3:1_4 arm

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)

# 2:1:2_4 arm

In [ ]:
# Replace with absolute path to root directory
input_directory = ""
# Replace with relative path to image directory within root directory
output_csv = ""

process_images(input_directory, output_csv)